In [3]:
import subprocess
import glob
import os
from os.path import exists
import re
import pandas as pd
from multiprocessing import Pool
import pyreadr

## NLRs

In [4]:
# Load the RDS file into Python
result = pyreadr.read_r(os.path.join('./NLRs_LRRs', "peakAnno_tb_DM1_DM2d_NLR_LRRs.rds"))
df = result[None]
df

,AGI,subClass,Class,seqnames,start,end,width,strand,Conc,Conc_DM1.DM2d,Conc_Col.0_12DAS,Fold,p.value,FDR,annotation,geneStrand,distanceToTSS,log2FC_RNA_DM1_DM2d
0,AT1G17750,LRR-RLKs,LRR,1,6105278,6105792,515,*,9.169747,8.973674,9.342332,-0.330542,2.302910e-04,8.147028e-04,Promoter (<=1kb),1,-531.0,0.221663
1,AT1G55610,LRR-RLKs,LRR,1,20786677,20787072,396,*,8.929118,9.337086,8.358192,0.906746,1.433535e-11,1.288136e-10,Promoter (2-3kb),2,-2660.0,5.013700
2,AT1G55610,LRR-RLKs,LRR,1,20784725,20785768,1044,*,10.439106,10.213131,10.634433,-0.394051,3.922709e-07,2.179612e-06,Promoter (<=1kb),2,-708.0,5.013700
3,AT1G51890,LRR-RLKs,LRR,1,19278556,19278730,175,*,7.950350,8.426414,7.234997,1.124701,7.317679e-21,1.231792e-19,Promoter (<=1kb),2,68.0,5.884220
4,AT1G51890,LRR-RLKs,LRR,1,19280170,19280481,312,*,8.406953,8.584910,8.203916,0.324014,9.571170e-04,2.955329e-03,Promoter (1-2kb),2,-1372.0,5.884220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,AT5G48620,CC-NBS-LRR,NLR,5,19714077,19714735,659,*,9.285059,9.408883,9.149601,0.218144,7.537352e-03,1.857587e-02,Promoter (1-2kb),1,-1369.0,0.429428
470,AT5G49140,TIR-NBS-LRR,NLR,5,19921957,19922295,339,*,8.656500,8.855653,8.425384,0.354583,2.092802e-03,5.970124e-03,Promoter (1-2kb),2,1120.0,2.013046
471,AT5G51630,TIR-NBS-LRR,NLR,5,20970901,20971353,453,*,9.082176,9.193800,8.961186,0.203603,3.644097e-03,9.774699e-03,Promoter (1-2kb),1,1773.0,3.356119
472,AT5G66900,RPW8-NBS-LRR,NLR,5,26717853,26718898,1046,*,10.683632,10.557705,10.799444,-0.225122,3.709744e-05,1.526959e-04,Promoter (<=1kb),2,0.0,1.184129


In [5]:
import pandas as pd
from Bio import SeqIO

def add_sequences_to_df(df, genome_path):
    # Load the genome fasta file
    genome_sequences = SeqIO.to_dict(SeqIO.parse(genome_path, "fasta"))

    # Create a column to store sequence data
    df['sequence'] = ''

    # Iterate over rows in the DataFrame
    for index, row in df.iterrows():
        # Extract sequence details
        seq_name = row['seqnames']
        start = int(row['start'])
        end = int(row['end'])
        
        # Get the corresponding sequence from the genome
        gene_seq = str(genome_sequences[seq_name].seq[start-1:end])

        # Assign the sequence to the 'sequence' column
        df.at[index, 'sequence'] = gene_seq

    return df

genome_path = os.path.join('/media/HDD2/Genomes/Ath_Ensembl56', 'Arabidopsis_thaliana.TAIR10.dna.toplevel.fa')

# Call the function with the DataFrame and genome path
df_with_sequences = add_sequences_to_df(df, genome_path)


In [6]:
df_with_sequences

,AGI,subClass,Class,seqnames,start,end,width,strand,Conc,Conc_DM1.DM2d,Conc_Col.0_12DAS,Fold,p.value,FDR,annotation,geneStrand,distanceToTSS,log2FC_RNA_DM1_DM2d,sequence
0,AT1G17750,LRR-RLKs,LRR,1,6105278,6105792,515,*,9.169747,8.973674,9.342332,-0.330542,2.302910e-04,8.147028e-04,Promoter (<=1kb),1,-531.0,0.221663,AGTCTTAGGATGTAGGTGGCCTTGTGATCTTGTGACAAAATGCAAT...
1,AT1G55610,LRR-RLKs,LRR,1,20786677,20787072,396,*,8.929118,9.337086,8.358192,0.906746,1.433535e-11,1.288136e-10,Promoter (2-3kb),2,-2660.0,5.013700,ACGCTTATATATATTTTGATAATTTTGTTTGACATTAAGGACCGAC...
2,AT1G55610,LRR-RLKs,LRR,1,20784725,20785768,1044,*,10.439106,10.213131,10.634433,-0.394051,3.922709e-07,2.179612e-06,Promoter (<=1kb),2,-708.0,5.013700,ACACCTAACCTATACAACTATACAAAGGAAATGAATATGATAAATT...
3,AT1G51890,LRR-RLKs,LRR,1,19278556,19278730,175,*,7.950350,8.426414,7.234997,1.124701,7.317679e-21,1.231792e-19,Promoter (<=1kb),2,68.0,5.884220,AGAGGAGGCTATTCATTTTTCATAGGTTCTTGGTGGGTAAAGATTG...
4,AT1G51890,LRR-RLKs,LRR,1,19280170,19280481,312,*,8.406953,8.584910,8.203916,0.324014,9.571170e-04,2.955329e-03,Promoter (1-2kb),2,-1372.0,5.884220,AGCTTAGAATGAACACGTTTGGTTGTTAATCTTTGTCTTTCTTCCT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,AT5G48620,CC-NBS-LRR,NLR,5,19714077,19714735,659,*,9.285059,9.408883,9.149601,0.218144,7.537352e-03,1.857587e-02,Promoter (1-2kb),1,-1369.0,0.429428,GGCACAGAGAGAATCGTGTAACAGATACGGTGCAGAATAATAGTAA...
470,AT5G49140,TIR-NBS-LRR,NLR,5,19921957,19922295,339,*,8.656500,8.855653,8.425384,0.354583,2.092802e-03,5.970124e-03,Promoter (1-2kb),2,1120.0,2.013046,AGACTAGTTGTGAGCCTTCCTAGTTCAAGTTTCCACCGATCCTTGC...
471,AT5G51630,TIR-NBS-LRR,NLR,5,20970901,20971353,453,*,9.082176,9.193800,8.961186,0.203603,3.644097e-03,9.774699e-03,Promoter (1-2kb),1,1773.0,3.356119,ACAACTATGGCATGAAGTTGAGTTGGGAAGAGCAGTTTCTGTCAGA...
472,AT5G66900,RPW8-NBS-LRR,NLR,5,26717853,26718898,1046,*,10.683632,10.557705,10.799444,-0.225122,3.709744e-05,1.526959e-04,Promoter (<=1kb),2,0.0,1.184129,CAATGTGTGGTTCATAAAAATGTAGATAGACGAAGGAGACGAATTG...


In [7]:
df_NLRs_up = df[(df['log2FC_RNA_DM1_DM2d'] > 1) & (df['Class'] == 'NLR')]
df_NLRs_up.shape

(84, 19)

In [8]:
df_NLRs_down = df[(df['log2FC_RNA_DM1_DM2d'] < -1 ) & (df['Class'] == 'NLR')]
df_NLRs_down.shape

(20, 19)

In [9]:
df

,AGI,subClass,Class,seqnames,start,end,width,strand,Conc,Conc_DM1.DM2d,Conc_Col.0_12DAS,Fold,p.value,FDR,annotation,geneStrand,distanceToTSS,log2FC_RNA_DM1_DM2d,sequence
0,AT1G17750,LRR-RLKs,LRR,1,6105278,6105792,515,*,9.169747,8.973674,9.342332,-0.330542,2.302910e-04,8.147028e-04,Promoter (<=1kb),1,-531.0,0.221663,AGTCTTAGGATGTAGGTGGCCTTGTGATCTTGTGACAAAATGCAAT...
1,AT1G55610,LRR-RLKs,LRR,1,20786677,20787072,396,*,8.929118,9.337086,8.358192,0.906746,1.433535e-11,1.288136e-10,Promoter (2-3kb),2,-2660.0,5.013700,ACGCTTATATATATTTTGATAATTTTGTTTGACATTAAGGACCGAC...
2,AT1G55610,LRR-RLKs,LRR,1,20784725,20785768,1044,*,10.439106,10.213131,10.634433,-0.394051,3.922709e-07,2.179612e-06,Promoter (<=1kb),2,-708.0,5.013700,ACACCTAACCTATACAACTATACAAAGGAAATGAATATGATAAATT...
3,AT1G51890,LRR-RLKs,LRR,1,19278556,19278730,175,*,7.950350,8.426414,7.234997,1.124701,7.317679e-21,1.231792e-19,Promoter (<=1kb),2,68.0,5.884220,AGAGGAGGCTATTCATTTTTCATAGGTTCTTGGTGGGTAAAGATTG...
4,AT1G51890,LRR-RLKs,LRR,1,19280170,19280481,312,*,8.406953,8.584910,8.203916,0.324014,9.571170e-04,2.955329e-03,Promoter (1-2kb),2,-1372.0,5.884220,AGCTTAGAATGAACACGTTTGGTTGTTAATCTTTGTCTTTCTTCCT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,AT5G48620,CC-NBS-LRR,NLR,5,19714077,19714735,659,*,9.285059,9.408883,9.149601,0.218144,7.537352e-03,1.857587e-02,Promoter (1-2kb),1,-1369.0,0.429428,GGCACAGAGAGAATCGTGTAACAGATACGGTGCAGAATAATAGTAA...
470,AT5G49140,TIR-NBS-LRR,NLR,5,19921957,19922295,339,*,8.656500,8.855653,8.425384,0.354583,2.092802e-03,5.970124e-03,Promoter (1-2kb),2,1120.0,2.013046,AGACTAGTTGTGAGCCTTCCTAGTTCAAGTTTCCACCGATCCTTGC...
471,AT5G51630,TIR-NBS-LRR,NLR,5,20970901,20971353,453,*,9.082176,9.193800,8.961186,0.203603,3.644097e-03,9.774699e-03,Promoter (1-2kb),1,1773.0,3.356119,ACAACTATGGCATGAAGTTGAGTTGGGAAGAGCAGTTTCTGTCAGA...
472,AT5G66900,RPW8-NBS-LRR,NLR,5,26717853,26718898,1046,*,10.683632,10.557705,10.799444,-0.225122,3.709744e-05,1.526959e-04,Promoter (<=1kb),2,0.0,1.184129,CAATGTGTGGTTCATAAAAATGTAGATAGACGAAGGAGACGAATTG...


In [10]:
def write_sequences_to_fasta(df, output_path):
    # Open the file in write mode
    with open(output_path, 'w') as fasta_file:
        # Iterate through the DataFrame
        for index, row in df.iterrows():
            # Construct the header using geneId, seqnames, start, and end
            # Replace 'geneId' with the appropriate column name for gene identifiers in your DataFrame
            header = f">{row['AGI']}_{row['seqnames']}_{row['start']}_{row['end']}"
            # Retrieve the sequence
            sequence = row['sequence']  # Ensure 'sequence' matches your DataFrame's column name
            
            # Write the header and sequence to the file
            fasta_file.write(f"{header}\n{sequence}\n")

output_path = './NLRs_LRRs/'
write_sequences_to_fasta(df, os.path.join(output_path, 'all_NLRs_LRRs_peaks.fasta'))


In [11]:
df_NLRs_down = df[(df['log2FC_RNA_DM1_DM2d'] < -1 ) & (df['Class'] == 'NLR')]
df_NLRs_up = df[(df['log2FC_RNA_DM1_DM2d'] > 1) & (df['Class'] == 'NLR')]

df_LRRs_up = df[(df['log2FC_RNA_DM1_DM2d'] > 1) & (df['Class'] == 'LRR')]
df_LRRs_down = df[(df['log2FC_RNA_DM1_DM2d'] < -1 ) & (df['Class'] == 'LRR')]
#print the nrow of each df
print(df_NLRs_down.shape)
print(df_NLRs_up.shape)
print(df_LRRs_down.shape)
print(df_LRRs_up.shape)

(20, 19)
(84, 19)
(142, 19)
(79, 19)


In [12]:
write_sequences_to_fasta(df_NLRs_down, os.path.join(output_path, 'all_NLRs_down_peaks.fasta'))
write_sequences_to_fasta(df_NLRs_up, os.path.join(output_path, 'all_NLRs_up_peaks.fasta'))

write_sequences_to_fasta(df_LRRs_down, os.path.join(output_path, 'all_LRRs_down_peaks.fasta'))
write_sequences_to_fasta(df_LRRs_up, os.path.join(output_path, 'all_LRRs_up_peaks.fasta'))

## prepare TIRs

In [31]:
# Load the RDS file into Python
TIRs = pyreadr.read_r(os.path.join('./TIRs', "peakAnno_tb_DM1_DM2d_TIRs.rds"))[None]
TIRs = TIRs.rename(columns={'geneId': 'AGI'})

In [32]:
TIRs = add_sequences_to_df(TIRs, genome_path)

In [33]:
TIRs

,seqnames,start,end,width,strand,Conc,Conc_DM1.DM2d,Conc_Col.0_12DAS,Fold,p.value,FDR,annotation,geneChr,geneStart,geneEnd,geneLength,geneStrand,AGI,distanceToTSS,sequence
0,1,21344814,21345638,825,*,10.270645,10.720302,9.613845,1.093734,4.445062e-75,6.314428e-73,Promoter (<=1kb),1,21345445,21346157,713,1,AT1G57630,0.0,TATCTGAACTGAACCAAATTGCAGAATTGTGACTTGAGAAAAACAG...
1,4,9495996,9496806,811,*,10.941896,10.536322,11.258116,-0.710798,1.370821e-46,7.652398e-45,Promoter (<=1kb),4,9488466,9495837,7372,2,AT4G16860,-159.0,AGTCTTAGTCTTTGTACAAGACAGAAATAATATTTCAATTTCTAGG...
2,4,9525707,9526712,1006,*,9.776423,10.245661,9.076541,1.139299,3.009211e-40,1.307940e-38,Promoter (<=1kb),4,9517950,9525800,7851,2,AT4G16920,0.0,TGAGATAGAATATCTTTAAAAGAAGTAAATCAGCAATACATAATGA...
3,1,27427728,27428464,737,*,9.704940,10.117510,9.124900,0.973040,3.086304e-36,1.121710e-34,Promoter (1-2kb),1,27429916,27432003,2088,1,AT1G72890,-1452.0,TTCCGTGTACAAAAGTAAATTAGAAAAATTTACAGAGAAATTTCAC...
4,4,9559878,9560086,209,*,9.540817,8.748015,10.049528,-1.254835,1.767202e-32,5.479661e-31,Promoter (1-2kb),4,9558571,9565569,6999,1,AT4G16990,1307.0,TCCTATATTATTTTGGGTGGTGTTGATGAGCCGTTGACGTAGAAGA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,5,18116728,18118709,1982,*,11.177318,11.114431,11.237578,-0.112480,2.058890e-02,4.444751e-02,Promoter (2-3kb),5,18114461,18119006,4546,1,AT5G44870,2267.0,TCTTTAGCAATATGTGCAATCTTCGATATCTCAAAATATACAGTTC...
134,5,18326356,18327288,933,*,10.166848,10.246298,10.082766,0.143400,2.082842e-02,4.488388e-02,Promoter (<=1kb),5,18326203,18332920,6718,1,AT5G45260,153.0,TCTCTGAAGCTCTCCGTCGAAAAGGCATAAATAATGTGGTCGTAGA...
135,5,18851768,18852699,932,*,9.875068,9.787873,9.957292,-0.149562,2.171653e-02,4.649379e-02,Promoter (1-2kb),5,18850645,18853896,3252,1,AT5G46490,1123.0,AAAGATAGATGATCCAACTGCACTGGAAGAGAGGCTAAAGTACCAA...
136,1,17363952,17365555,1604,*,10.944029,11.026880,10.856128,0.148811,2.209196e-02,4.717185e-02,Distal Intergenic,1,17365898,17368703,2806,2,AT1G47370,3148.0,GCTCCAAGATATAAAGCACGGTTGGTAGCAAAGGGATTCACACAGA...


In [34]:
TIR_common_up = ['AT1G09665','AT1G57670','AT1G57830','AT1G60320','AT2G03030','AT2G03300','AT4G04110','AT4G19910','AT1G27170','AT1G27180','AT1G63730','AT1G63880','AT1G69550','AT1G72840','AT4G09430','AT4G12010','AT4G16900','AT4G16930','AT4G16950','AT4G16990','AT4G19500','AT4G19510','AT4G19530','AT5G17680','AT5G17880','AT5G17890','AT5G36930','AT5G38850','AT5G40090','AT5G40100','AT5G45050','AT5G45060','AT5G45210','AT5G45220','AT5G45230','AT5G45240','AT5G45260','AT5G46270','AT5G46450','AT5G48770']


TIR_not_up = ['AT1G51270','AT1G57630','AT1G72900','AT5G46520','AT5G45000','AT5G41750','AT5G45090','AT3G04220','AT3G44400','AT2G20142','AT2G32140','AT2G16870','AT4G11170','AT4G14370','AT1G66090','AT4G19520','AT1G72940','AT1G72920']

In [35]:
df_TIR_common_up = TIRs[TIRs['AGI'].isin(TIR_common_up)]
df_TIR_not_up = TIRs[TIRs['AGI'].isin(TIR_not_up)]

In [36]:
write_sequences_to_fasta(df_TIR_not_up, os.path.join('TIRs/', 'df_TIR_not_up.fasta'))
write_sequences_to_fasta(df_TIR_common_up, os.path.join('TIRs/', 'df_TIR_common_up.fasta'))

In [43]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/TIRs/common_up_TIRs_vs_not_up" --control "/data/TIRs/df_TIR_not_up.fasta" \
  "/data/TIRs/df_TIR_common_up.fasta" "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"

Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/TIRs/common_up_TIRs_vs_not_up'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (36).
MOTIF: 1 SEQ: 53/53
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 53/53


In [44]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/TIRs/common_up_TIRs_shuffle" --control --shuffle-- \
  "/data/TIRs/df_TIR_common_up.fasta" "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"

Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/TIRs/common_up_TIRs_shuffle'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Creating control sequences by shuffling input sequences preserving 2-mers.
Not in partition maximization mode. Fixing partition at the number of primary sequences (36).
MOTIF: 1 SEQ: 1044/1044
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 1044/1044


In [41]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_LRRs_up_vs_down" --control "/data/NLRs_LRRs/all_LRRs_down_peaks.fasta" \
  "/data/NLRs_LRRs/all_LRRs_up_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"

Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
The output directory '/data/NLRs_LRRs/all_LRRs_up_vs_down' already exists.
Its contents will be overwritten.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (79).
MOTIF: 1 SEQ: 221/221
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 221/221


In [59]:
TIRs_ame = pd.read_csv('TIRs/common_up_TIRs_shuffle/ame.tsv', sep='\t')
TIRs_ame =  TIRs_ame[TIRs_ame['consensus'].notna()]
TIRs_ame

,rank,motif_DB,motif_ID,motif_alt_ID,consensus,p-value,adj_p-value,E-value,tests,FASTA_max,pos,neg,PWM_min,TP,%TP,FP,%FP
0,1,/data/scan_motif/Ath_TF_binding_motifs_plantTF...,AT1G49480,MP00192,RAMAAAAAAAAAAAA,2.590000e-08,0.000004,0.00224,140.0,36.0,36.0,1008.0,5.27,13.0,36.11,49.0,4.86
1,2,/data/scan_motif/Ath_TF_binding_motifs_plantTF...,AT2G45660,MP00076,TTTYTYTTTYTYTYTYTTTYT,7.010000e-07,0.000217,0.13400,309.0,36.0,36.0,1008.0,16.90,10.0,27.78,34.0,3.37
2,3,/data/scan_motif/Ath_TF_binding_motifs_plantTF...,AT1G72050,MP00229,CYTCCTCCTCCTCCTCCTC,1.580000e-06,0.000220,0.13600,139.0,36.0,36.0,1008.0,53.70,6.0,16.67,7.0,0.69
3,4,/data/scan_motif/Ath_TF_binding_motifs_plantTF...,AT5G17430,MP00610,AADAAARARAAAAARAGARA,2.230000e-06,0.000556,0.34400,250.0,36.0,36.0,1008.0,3.07,17.0,47.22,137.0,13.59
4,5,/data/scan_motif/Ath_TF_binding_motifs_plantTF...,AT5G63090,MP00581,WCCGCCGCCDYCKCCGCCGCH,7.580000e-05,0.002800,1.73000,37.0,36.0,36.0,1008.0,17.20,4.0,11.11,4.0,0.40


In [60]:
# map_id_name <- read.csv('../gene_table_and_other_scripts/bioMartR_Ath_all_genes_info_googled.csv')
map_id_name = pd.read_csv('../gene_table_and_other_scripts/bioMartR_Ath_all_genes_info_googled.csv')
map_id_name

,ensembl_gene_id,external_gene_name
0,AT5G16970,AER
1,AT4G32100,AT4G32100
2,AT2G43120,AT2G43120
3,AT1G30814,AT1G30814
4,AT3G18710,PUB29
...,...,...
32828,AT3G14120,NUP107
32829,AT3G28630,CROLIN1
32830,AT1G23140,CAR8
32831,AT5G09780,REM25


In [61]:
# Performing a left join
TIRs_ame = pd.merge(TIRs_ame, map_id_name, left_on='motif_ID', right_on='ensembl_gene_id', how='left')

In [67]:
TIRs_ame.to_csv('TIRs/common_up_TIRs_shuffle/ame.tsv', sep='\t', index=False)

In [68]:
TIRs_ame.to_csv('TIRs/common_up_TIRs_shuffle/ame.csv', sep=',', index=False)


### MEME installed from conda seems to be broken, use docker version instead

In [80]:
! sudo chmod -R 777 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/WGCNA_ATAC_integration/


[sudo] password for hu: 


In [34]:
! sudo chmod -R 777 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/NLRs_LRRs

[sudo] password for hu: 


docker takes sudo, can run visudo and add "yourusername ALL=(ALL) NOPASSWD: /usr/bin/docker"
then run the script docker_mem_ame.sh, which loops over all fasta files

In [146]:
sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/:/data memesuite/memesuite ame --oc /data/WGCNA_ATAC_integration/ame_WGCNA_vs_non_variable_peaks/ --control /data/WGCNA_ATAC_integration/fasta_files_and_control/non_variable_peaks_in_RNA_seq.fasta /data/WGCNA_ATAC_integration/fasta_files_and_control/all_WGCNA_seq.fasta /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme

SyntaxError: invalid syntax (1206664557.py, line 1)

/media/HDD2/donghui/bulk_ATAC_DM1_DM2d » mkdir WGCNA_ATAC_integration/ame_results_with_shuffle/                      hu@chaelab2
(work2) -------------------------------------------------------------------------------------------------------------------------
/media/HDD2/donghui/bulk_ATAC_DM1_DM2d » chmod -R 777 WGCNA_ATAC_integration/ame_results_with_shuffle/   
/media/HDD2/donghui/bulk_ATAC_DM1_DM2d » bash ./docker_mem_ame_with_shuffle.sh    

In [39]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_LRRs_up" --control --shuffle-- \
  "/data/NLRs_LRRs/all_LRRs_up_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"


Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/NLRs_LRRs/all_LRRs_up'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Creating control sequences by shuffling input sequences preserving 2-mers.
Not in partition maximization mode. Fixing partition at the number of primary sequences (79).
MOTIF: 1 SEQ: 1106/1106
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 1106/1106


In [40]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_LRRs_down" --control --shuffle-- \
  "/data/NLRs_LRRs/all_LRRs_down_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"


Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/NLRs_LRRs/all_LRRs_down'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Creating control sequences by shuffling input sequences preserving 2-mers.
Not in partition maximization mode. Fixing partition at the number of primary sequences (142).
MOTIF: 1 SEQ: 1278/1278
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 1278/1278


In [45]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_NLRs_up" --control --shuffle-- \
  "/data/NLRs_LRRs/all_NLRs_up_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"


Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
The output directory '/data/NLRs_LRRs/all_NLRs_up' already exists.
Its contents will be overwritten.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Creating control sequences by shuffling input sequences preserving 2-mers.
Not in partition maximization mode. Fixing partition at the number of primary sequences (84).
MOTIF: 1 SEQ: 1092/1092
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 1092/1092


In [42]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_NLRs_down" --control --shuffle-- \
  "/data/NLRs_LRRs/all_NLRs_down_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"


Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/NLRs_LRRs/all_NLRs_down'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Creating control sequences by shuffling input sequences preserving 2-mers.
Not in partition maximization mode. Fixing partition at the number of primary sequences (20).
MOTIF: 1 SEQ: 1040/1040
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 1040/1040


In [44]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_NLRs_up_vs_down" --control "/data/NLRs_LRRs/all_LRRs_down_peaks.fasta" \
  "/data/NLRs_LRRs/all_NLRs_up_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"

Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/NLRs_LRRs/all_NLRs_up_vs_down'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (84).
MOTIF: 1 SEQ: 226/226
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 226/226


In [48]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_NLRs_down_vs_up" --control "/data/NLRs_LRRs/all_LRRs_up_peaks.fasta" \
  "/data/NLRs_LRRs/all_NLRs_down_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"

Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/NLRs_LRRs/all_NLRs_down_vs_up'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (20).
MOTIF: 1 SEQ: 99/99
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 99/99


In [46]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_LRRs_up_vs_down" --control "/data/NLRs_LRRs/all_LRRs_down_peaks.fasta" \
  "/data/NLRs_LRRs/all_LRRs_up_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"

Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/NLRs_LRRs/all_LRRs_up_vs_down'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (79).
MOTIF: 1 SEQ: 221/221
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 221/221


In [47]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  ame --oc "/data/NLRs_LRRs/all_LRRs_down_vs_up" --control "/data/NLRs_LRRs/all_LRRs_up_peaks.fasta" \
  "/data/NLRs_LRRs/all_LRRs_down_peaks.fasta" \
  "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme"

Added /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme to motif_sources which now has 1 file names.
Motif file name is /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme.
Writing results to output directory '/data/NLRs_LRRs/all_LRRs_down_vs_up'.
E-value threshold for reporting results: 10
Checking alphabets in 1 motif files.
Loading motifs from file '/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (142).
MOTIF: 1 SEQ: 221/221
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 619 SEQ: 221/221


In [53]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  fimo --oc "/data/NLRs_LRRs/fimo_LRRs_up" \
    "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme" "/data/NLRs_LRRs/all_LRRs_up_peaks.fasta" 

Using motif +AT1G01060 of width 10.
Using motif -AT1G01060 of width 10.
Finding best site passing the output threshold in each of the 79 sequences.
Found a best site in 19 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set of 10000 p-values.
#   Estimating pi_0.
# Minimal pi_zero = 0.863
#   Estimated pi_0=0.866078
Using motif +AT1G01250 of width 15.
Using motif -AT1G01250 of width 15.
Finding best site passing the output threshold in each of the 79 sequences.
Found a best site in 4 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set of 10000 p-values.
#   Estimating pi_0.
# Minimal pi_zero = 1.00191
#   Estimated pi_0=1
Using motif +AT1G01260 of width 8.
Using motif -AT1G01260 of width 8.
Finding best site passing the output threshold in each of the 79 sequences.
Found a best site in 3 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set

In [54]:
! sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d:/data memesuite/memesuite \
  fimo --oc "/data/NLRs_LRRs/fimo_NLRs_up" \
    "/data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme" "/data/NLRs_LRRs/all_NLRs_up_peaks.fasta"

Using motif +AT1G01060 of width 10.
Using motif -AT1G01060 of width 10.
Finding best site passing the output threshold in each of the 84 sequences.
Found a best site in 24 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set of 10000 p-values.
#   Estimating pi_0.
# Minimal pi_zero = 0.900784
#   Estimated pi_0=0.904
Using motif +AT1G01250 of width 15.
Using motif -AT1G01250 of width 15.
Finding best site passing the output threshold in each of the 84 sequences.
Found a best site in 8 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set of 10000 p-values.
#   Estimating pi_0.
# Minimal pi_zero = 1.00302
#   Estimated pi_0=1
Using motif +AT1G01260 of width 8.
Using motif -AT1G01260 of width 8.
Finding best site passing the output threshold in each of the 84 sequences.
Found a best site in 5 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set